In [1]:
from dataclasses import dataclass
from diffusers import UNet2DModel
from diffusers import DDPMScheduler
import torch
from accelerate import Accelerator
from diffusers import DDPMPipeline
from dataclasses import dataclass
from diffusers import DDPMPipeline
from PIL import Image
import os

@dataclass
class TrainingConfig:
    image_size = 32  # the generated image resolution
    seed = 24
    mixed_precision = "fp16"
    learning_rate = 1e-4
    lr_warmup_steps = 500
    gradient_accumulation_steps = 1
    image_number= 64 #Number of test images
    generated_images_output=""
    model_path=""

config = TrainingConfig()

# Build U-net from saved weights

In [2]:
Unet = UNet2DModel.from_pretrained(config.model_path)
accelerator = Accelerator(
    mixed_precision=config.mixed_precision,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    log_with="tensorboard",
    project_dir=os.path.join("/artifacts/temp", "logs"),
)
Unet = Unet.to(accelerator.device)
noise_scheduler = DDPMScheduler(num_train_timesteps=1000)
pipeline = DDPMPipeline(unet=accelerator.unwrap_model(Unet), scheduler=noise_scheduler)

# Generate images

In [3]:
def evaluate(config, pipeline):
    # Sample some images from random noise (this is the backward diffusion process).
    # The default pipeline output type is `List[PIL.Image]`
    images = pipeline(
        batch_size=config.image_number,
        generator=torch.manual_seed(config.seed),
    ).images

    return images

In [ ]:
images=evaluate(config, pipeline)

In [5]:
#  Create the output directory if itdoesn't exist
if not os.path.exists(config.generated_images_output):
    os.makedirs(config.generated_images_output)

# Iterate over the images and save each one
for i, img in enumerate(images):
    # Construct the file path for each image
    file_path = os.path.join(config.generated_images_output, f"image_{i}.png")
    img.save(file_path)